<a href="https://colab.research.google.com/github/MaricelaMH/SIMULACION-II/blob/main/Linea_de_espera_combi_proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SIMULACIÓN DE UNA PARADA DE COMBIS

Contexto del problema: En Naucalpan de Juarez, Estado de México, hay una parada de combis en el paradero de Cuatro Caminos donde los pasajeros se forman para esperar la próxima llegada de una combi. Esta parada sigue una disciplina de servicio primero en entrar, primero en salir (FIFO), donde la combi recoge a los pasajeros en el orden en que llegaron. Para ello realizaremos una simulación para lograr entender mejor el comportamiento de nuestro sistema y asi tomar desiciones sobre los horarios y capacidad de las combis para evitar tiempos de esperas excesivos parea los pasajeros.

1. Para simular lo anterior, usaremos la libreria simpy por lo que la importaremos junto con las demas librerias que usaremos.

In [1]:
!pip install simpy

In [136]:
import matplotlib.pyplot as plt
import numpy as np
from random import random, seed
import pandas as pd
import simpy

2. Definimos nuestros parámetros

In [188]:
# Parámetros del sistema de combis
no_combis = 1            # Cantidad de combis disponibles en la parada
tc_min = 1                  # Tiempo mínimo de abordaje por pasajero (minutos)
tc_max = 2                  # Tiempo máximo de abordaje por pasajero (minutos)
mu = 5                 # Tasa de llegada de los pasajeros (1 cada 2 minutos)
ll =  20                 # Tasa de llegada de combis (1 cada 15 minutos)
pasajeros = 50              # Total de pasajeros a simular
cifras = 2                  # Cantidad de cifras decimales para mostrar
cap_combi = 19              # Capacidad máxima de pasajeros de cada combi

In [189]:
TLL, TA, TS = [], [], []   # Listas para tiempos de llegada de los pasajeros, abordaje, y tiempo de salida de la combi

In [190]:
# Parámetro de simulación
semilla = 13
seed(semilla)

3. Creamos una función que nos ayudara a simular la llegada de los pasajeros a la parada de combis las cuales suelen seguir una distribución exponencial, por lo que la media estara definida como:

$$ \mu = \frac{1}{ll}  $$

donde $ll$ es el tiempo promedio de llegada de los pasajeros

In [191]:
# Función para simular cuando los pasajeros llegan a la parada de combis.
def parada_combis(env, parada, num_pasajeros, texto=True):
    for i in range(num_pasajeros):
        # Tiempo hasta la próxima llegada del pasajero siguiendo distribución exponencial
        tll = np.random.exponential(scale=mu)
        yield env.timeout(tll) #Pausamos la ejecución durante tll unidades de tiempo
        TLL.append(env.now)  # Almacenamos el tiempo de llegada del pasajero
        if texto:
            print(f'----Ha llegado el pasajero número {i} en el minuto {round(env.now, cifras)}----')
        env.process(abordaje(env, parada, i, texto))

4. Creamos nuevamente otra función que nos ayudara a simular el

In [192]:
def llegada_combis(env, parada, texto=True):
   numero_combi = 1  # Contador para identificar cada combi
   while True:
        # Tiempo entre llegadas de combis siguiendo distribución exponencial
        t_llegada_combis = np.random.exponential(scale=ll)
        yield env.timeout(t_llegada_combis)

        num_p = len(parada.queue)  # Pasajeros esperando en la cola
        if texto:
            print(f'La combi número {numero_combi} llega a la parada a los {round(env.now, 2)} minutos')

        # Si hay pasajeros, determinamos cuantos pasajeros recojera la combi
        if num_p > 0:
            pasajeros_a_recoger = min(num_p,cap_combi)
            for _ in range(pasajeros_a_recoger):
                parada.release(parada.request())  # Se simula la subida de los pasajeros al autobús
            if texto:
                print(f'La combi recoge a {pasajeros_a_recoger} pasajeros en la parada a los {round(env.now, cifras)} minutos')

        # Si quedan pasajeros en la cola después de llenar la combi, esperan a la siguiente
            if num_p > cap_combi:
                if texto:
                    print(f'Quedan {num_p - 19} pasajeros esperando la próxima combi en la parada a los {round(env.now, 2)} minutos')

            # Agregar el mensaje indicando que la combi ha salido
            if texto:
                print(f'La combi número {numero_combi} sale del paradero con {pasajeros_a_recoger} pasajeros a los {round(env.now, 2)} minutos')

        # Incrementar el número de combi para la siguiente llegada
        numero_combi += 1

5. Creamos una función llamada abordaje la cual ayudara a simular el tiempo que le lleva al pasajero abordar una combi , donde utilizaremos una distribución uniforme para determinar el tiempo mínimo y máximo de abordaje del pasajero.

In [193]:
def abordaje(env, parada, num, texto=True):
    global TLL,TA,TS
    # Solicitamos abordar una combi
    with parada.request() as request:  # Solicitamos una combi
        yield request  # Hace que el pasajero espere hasta que pueda abordar la combi
        # Tiempo de abordaje
        tc = np.random.uniform(tc_min, tc_max)  # Tiempo de corte aleatorio
        TA.append(tc)
        if texto:
            print(f'**** El pasajero {num} inicia el abordaje en el minuto {round(env.now, cifras)}****')
        yield env.timeout(tc)
        TS.append(env.now)
        if texto:
            print(f'xxx El pasajero {num} termina el abordaje en el minuto {round(env.now, cifras)}xxx')


In [194]:
# prompt: como evito que se generen muchos resultados al ejecutar el codigo anterior

def exp(n):
    long_cola = []
    tiempo_esp = []
    uso = [] # Porcentaje de uso de las combis
    combis_necesarias_totales = []

    for _ in range(n):
        global TLL, TA, TS
        TLL, TA, TS = [], [], []

        env.process(parada_combis(env, parada, pasajeros, texto=True)) # Set texto to False to suppress printing
        env.process(llegada_combis(env, parada, texto=True)) # Set texto to False to suppress printing
        env.run(until=120) # Tiempo de simulación (ajustado)

        TLL, TA, TS = np.array(TLL), np.array(TA), np.array(TS)
        num_abordos = len(TS)
        TLL = TLL[:num_abordos]
        TA = TA[:num_abordos]
        t_esp = sum(TS - TLL - TA)
        fin_sim = env.now

        long_cola.append(t_esp / fin_sim)
        tiempo_esp.append(t_esp / pasajeros)
        uso.append(100 * sum(TA) / fin_sim)

        # Calcular el número total de combis necesarias
        # Basado en la cantidad total de pasajeros en cola y la capacidad del autobús
        total_pasajeros = len(TLL)  # Total de pasajeros atendidos
        total_combis = min(np.ceil(total_pasajeros / cap_combi),no_combis)  # Combis necesarias
        combis_necesarias_totales.append(total_combis)


        return long_cola, tiempo_esp, uso, combis_necesarias_totales  # Asegurarse de que se devuelven cuatro valores

In [195]:
env = simpy.Environment()
parada = simpy.Resource(env, capacity=no_combis)
long_cola, tiempo_esp, uso, combis_necesarias_totales = exp(10) #Example call to exp


----Ha llegado el pasajero número 0 en el minuto 0.72----
**** El pasajero 0 inicia el abordaje en el minuto 0.72****
----Ha llegado el pasajero número 1 en el minuto 1.15----
xxx El pasajero 0 termina el abordaje en el minuto 2.48xxx
**** El pasajero 1 inicia el abordaje en el minuto 2.48****
xxx El pasajero 1 termina el abordaje en el minuto 3.94xxx
----Ha llegado el pasajero número 2 en el minuto 12.09----
**** El pasajero 2 inicia el abordaje en el minuto 12.09****
xxx El pasajero 2 termina el abordaje en el minuto 13.56xxx
----Ha llegado el pasajero número 3 en el minuto 14.58----
**** El pasajero 3 inicia el abordaje en el minuto 14.58****
xxx El pasajero 3 termina el abordaje en el minuto 16.4xxx
----Ha llegado el pasajero número 4 en el minuto 17.26----
**** El pasajero 4 inicia el abordaje en el minuto 17.26****
----Ha llegado el pasajero número 5 en el minuto 17.51----
xxx El pasajero 4 termina el abordaje en el minuto 18.35xxx
**** El pasajero 5 inicia el abordaje en el minu